# 0. Install packages

In [ ]:
options(repos = c(CRAN = "https://cloud.r-project.org"))
Sys.setenv(R_FORCE_BIOCINSTALLER = "FALSE")

userlib <- path.expand("~/Rlibs")
dir.create(userlib, showWarnings = FALSE, recursive = TRUE)
.libPaths(c(userlib, .libPaths()))

if (!requireNamespace("pak", quietly = TRUE)) install.packages("pak")

pkgs <- c(
  "systemfonts","textshaping","svglite",
  "ggplot2","dplyr","tidyr",
  "survival","survminer",
  "Hmisc","boot","gridExtra","cowplot","pROC",
  "patchwork","ggbreak","scales"
)

pak::pak(pkgs)

# 1. download necessary data

## 1.1 outcomes

In [ ]:
system("dx download 'UKBRISK/to_event_Touchscreen_v3.tsv'")
endpoints <- read.delim("to_event_Touchscreen_v3.tsv", sep = "\t")

### 1.1.1 followup 10y-censoring for nhc endpoints

In [ ]:
endpoints_nhc <- c("CVD", "RD", "DM")

endpoints_10y <- endpoints

for (endpoint in endpoints_nhc) {
  status_col <- paste0(endpoint, "_status")
  followup_col <- paste0(endpoint, "_followup")
  endpoints_10y[[status_col]][endpoints_10y[[followup_col]] > 10] <- FALSE
}

for (endpoint in endpoints_nhc) {
  followup_col <- paste0(endpoint, "_followup")
  endpoints_10y[[followup_col]] <- pmin(endpoints_10y[[followup_col]], 10)
}

## 1.2 Risk predictions

### 1.2.1 Benchmarking LPs

In [ ]:
dir <- "UKBRISK_ENModels/Benchmarking"
files <- system(paste0("dx ls ", dir), intern = TRUE)

lp_files <- files[grepl("_train_LP.tsv|_test_LP.tsv", files)]


In [ ]:
merged_train_dataframes <- list()
merged_test_dataframes <- list()

for (lp_file in lp_files) {
  download_cmd <- paste("dx download", paste0(dir, "/", lp_file))
  system(download_cmd)
    
  lp_data <- read.csv(lp_file, sep = "\t", header = TRUE)
  
  parts <- strsplit(lp_file, "_")[[1]]
  endpoint <- parts[1]
  combo_name <- paste(parts[2:(length(parts) - 2)], collapse = "_")
  set_type <- parts[length(parts) - 1]  # "train" or "test"
  
  column_name <- combo_name
  
  #train
  if (set_type == "train") {
    if (!is.null(merged_train_dataframes[[endpoint]])) {
      merged_train_dataframes[[endpoint]] <- merge(
        merged_train_dataframes[[endpoint]], 
        lp_data[, c("eid", "LP")], 
        by = "eid", 
        all = TRUE
      )
      colnames(merged_train_dataframes[[endpoint]])[ncol(merged_train_dataframes[[endpoint]])] <- column_name
    } else {
      lp_data_train <- lp_data
      colnames(lp_data_train)[2] <- column_name
      merged_train_dataframes[[endpoint]] <- lp_data_train
    }
  }
  
  #test
  if (set_type == "test") {
    if (!is.null(merged_test_dataframes[[endpoint]])) {
      merged_test_dataframes[[endpoint]] <- merge(
        merged_test_dataframes[[endpoint]], 
        lp_data[, c("eid", "LP")], 
        by = "eid", 
        all = TRUE
      )
      colnames(merged_test_dataframes[[endpoint]])[ncol(merged_test_dataframes[[endpoint]])] <- column_name
    } else {
      lp_data_test <- lp_data
      colnames(lp_data_test)[2] <- column_name
      merged_test_dataframes[[endpoint]] <- lp_data_test
    }
  }
}

In [ ]:
#merge with endpoints

for (endpoint in names(merged_train_dataframes)) {
  merged_train_dataframes[[endpoint]] <- merge(
    merged_train_dataframes[[endpoint]], 
    endpoints[, c("eid", paste0(endpoint, "_followup"), paste0(endpoint, "_status"))], 
    by = "eid", 
    all.x = TRUE
  )
}

for (endpoint in names(merged_test_dataframes)) {
  merged_test_dataframes[[endpoint]] <- merge(
    merged_test_dataframes[[endpoint]], 
    endpoints[, c("eid", paste0(endpoint, "_followup"), paste0(endpoint, "_status"))], 
    by = "eid", 
    all.x = TRUE
  )
}

In [ ]:
# check that everything is there
for (endpoint in names(merged_test_dataframes)) {
  
  selected_columns <- c("eid", "pmh", "pmh_ts", "prs_metabolomics_pmh_ts", 
                        "clinicalrisk", "everything", "score", "qrisk", "prevent", paste0(endpoint, "_status"), 
                        paste0(endpoint, "_followup"))
  
  actual_columns <- colnames(merged_test_dataframes[[endpoint]])
  
  missing_columns <- selected_columns[!selected_columns %in% actual_columns]
  
  if (length(missing_columns) > 0) {
    cat("For endpoint", endpoint, "the following columns are missing:", missing_columns, "\n")
  } else {
    cat("For endpoint", endpoint, "all selected columns are present.\n")
  }
}


### 1.2.2 CV absolute 10y risk

In [ ]:
dir <- "UKBRISK_ENModels/Benchmarking"
files <- system(paste0("dx ls ", dir), intern = TRUE)

absrisk_files <- files[grepl("_combined_10y.csv", files)]

for (file_name in absrisk_files) {
  download_cmd <- paste0("dx download ", dir, "/", file_name, " --overwrite")
  system(download_cmd)
}

merged_dataframes <- list()

for (file_name in absrisk_files) {
    
  abs_risk_data <- read.csv(file_name, header = TRUE)
  
  parts <- strsplit(file_name, "_")[[1]]
  
  endpoint <- parts[2] 
  
  survival_index <- which(parts == "survival")[1] - 1 
  
  combo_name <- paste(parts[3:survival_index], collapse = "_")
  
  if (!is.null(merged_dataframes[[endpoint]])) {

    merged_dataframes[[endpoint]] <- merge(
      merged_dataframes[[endpoint]], 
      abs_risk_data[, c("eid", "survival_probability", "set")], 
      by = c("eid", "set"), 
      all = TRUE
    )

    colnames(merged_dataframes[[endpoint]])[ncol(merged_dataframes[[endpoint]])] <- combo_name
  } else {

    abs_risk_data_merged <- abs_risk_data[, c("eid", "set", "survival_probability")]
    colnames(abs_risk_data_merged)[3] <- combo_name
    merged_dataframes[[endpoint]] <- abs_risk_data_merged
  }
}

cat("All absolute risk data frames merged successfully.\n")

for (endpoint in names(merged_dataframes)) {
  merged_dataframes[[endpoint]] <- merge(
    merged_dataframes[[endpoint]], 
    endpoints_10y[, c("eid", paste0(endpoint, "_followup"), paste0(endpoint, "_status"))], 
    by = "eid", 
    all.x = TRUE
  )
}

train_dataframes_absrisk_cv_10y <- list()
test_dataframes_absrisk_cv_10y <- list()


for (endpoint in c("CVD", "PAD", "ISS", "CAD", "HF")) {
  df <- merged_dataframes[[endpoint]]
  
  
  train_dataframes_absrisk_cv_10y[[endpoint]] <- df[df$set == "train", ]
  test_dataframes_absrisk_cv_10y[[endpoint]] <- df[df$set == "test", ]
}

### 1.2.3 Download BM coefficients

In [ ]:
dir <- "UKBRISK_ENModels/Benchmarking"

files <- system(paste0("dx ls ", dir), intern = TRUE)

coef_files <- files[grepl("_coefficients.tsv", files)]

merged_coef_dataframes <- list()

for (coef_file in coef_files) {

  download_cmd <- paste("dx download", paste0(dir, "/", coef_file))
  system(download_cmd)
  

  coef_data <- read.csv(coef_file, sep = "\t", header = TRUE, row.names = 1)
  
  
  coef_data$Predictor <- rownames(coef_data)
  

  parts <- strsplit(coef_file, "_")[[1]]
  endpoint <- parts[1]
  combo_name <- paste(parts[2:(length(parts) - 1)], collapse = "_")  
  
  column_name <- combo_name
  

  if (!is.null(merged_coef_dataframes[[endpoint]])) {
    merged_coef_dataframes[[endpoint]] <- merge(
      merged_coef_dataframes[[endpoint]], 
      coef_data[, c("Predictor", "Coefficient")], 
      by = "Predictor", 
      all = TRUE
    )
    colnames(merged_coef_dataframes[[endpoint]])[ncol(merged_coef_dataframes[[endpoint]])] <- column_name
  } else {
    coef_data_merged <- coef_data[, c("Predictor", "Coefficient")]
    colnames(coef_data_merged)[2] <- column_name
    merged_coef_dataframes[[endpoint]] <- coef_data_merged
  }
}


In [ ]:
for (endpoint in names(merged_coef_dataframes)) {

  df <- merged_coef_dataframes[[endpoint]]
  
  df$Category <- sapply(df$Predictor, function(x) strsplit(x, "_")[[1]][1])

  merged_coef_dataframes[[endpoint]] <- df
}

In [ ]:
for (endpoint in names(merged_coef_dataframes)) {

  df <- merged_coef_dataframes[[endpoint]]
  
  df$Pred_clean <- sapply(df$Predictor, function(x) sub("^[^_]*_", "", x))
  
  merged_coef_dataframes[[endpoint]] <- df
}


In [ ]:
system("dx download 'UKBRISK_Processed/mapping_touchscreen_category.tsv'")
mapping_tscat <- read.delim("mapping_touchscreen_category.tsv", sep = "\t")

In [ ]:
names(mapping_tscat)[names(mapping_tscat) == "Category"] <- "ts_cat"
names(mapping_tscat)[names(mapping_tscat) == "Sub.category"] <- "ts_subcat"

In [ ]:
head(mapping_tscat)

In [ ]:
for (endpoint in names(merged_coef_dataframes)) {

  df <- merged_coef_dataframes[[endpoint]]
  
  df <- merge(df, mapping_tscat[, c("Column.names", "ts_cat", "ts_subcat")], 
              by.x = "Pred_clean", by.y = "Column.names", all.x = TRUE)
  
  merged_coef_dataframes[[endpoint]] <- df
}


In [ ]:
target_categories <- c('clinicalrisk_Age.at.recruitment', 
                       'qrisk_Age.at.recruitment',
                       'prevent_Age.at.recruitment',
                       'score_Age.at.recruitment',
                       'clinicalrisk_Sex_0',
                       'prevent_Sex_0',
                       'qrisk_Sex_0',
                       'score_Sex_0',
                       'clinicalrisk_Sex_1',
                       'prevent_Sex_1',
                       'score_Sex_1',
                       'qrisk_Sex_1')

for (endpoint in names(merged_coef_dataframes)) {
    df <- merged_coef_dataframes[[endpoint]]
    
    df$Category <- ifelse(df$Predictor %in% target_categories, "agesex", df$Category)
    
    merged_coef_dataframes[[endpoint]] <- df
}


### 1.2.4 Step 1: Absolute Risk 10y NHC

In [ ]:
dir <- "UKBRISK_ENModels/NHC/Initial_10y"
files <- system(paste0("dx ls ", dir), intern = TRUE)

absrisk_files <- files[grepl("_combined_10y.csv", files)]

for (file_name in absrisk_files) {
  download_cmd <- paste0("dx download ", dir, "/", file_name, " --overwrite")
  system(download_cmd)
}

merged_dataframes <- list()

for (file_name in absrisk_files) {
    
  abs_risk_data <- read.csv(file_name, header = TRUE)
  
  parts <- strsplit(file_name, "_")[[1]]
  
  endpoint <- parts[2]  

  survival_index <- which(parts == "survival")[1] - 1  
  
  combo_name <- paste(parts[3:survival_index], collapse = "_")
  
  if (!is.null(merged_dataframes[[endpoint]])) {
 
    merged_dataframes[[endpoint]] <- merge(
      merged_dataframes[[endpoint]], 
      abs_risk_data[, c("eid", "survival_probability", "set")], 
      by = c("eid", "set"), 
      all = TRUE
    )
    colnames(merged_dataframes[[endpoint]])[ncol(merged_dataframes[[endpoint]])] <- combo_name
  } else {

    abs_risk_data_merged <- abs_risk_data[, c("eid", "set", "survival_probability")]
    colnames(abs_risk_data_merged)[3] <- combo_name
    merged_dataframes[[endpoint]] <- abs_risk_data_merged
  }
}

for (endpoint in names(merged_dataframes)) {
  merged_dataframes[[endpoint]] <- merge(
    merged_dataframes[[endpoint]], 
    endpoints_10y[, c("eid", paste0(endpoint, "_followup"), paste0(endpoint, "_status"))], 
    by = "eid", 
    all.x = TRUE
  )
}

train_dataframes_absrisk_nhc_10y <- list()
test_dataframes_absrisk_nhc_10y <- list()


for (endpoint in names(merged_dataframes)) {
  df <- merged_dataframes[[endpoint]]

  train_dataframes_absrisk_nhc_10y[[endpoint]] <- df[df$set == "train", ]
  test_dataframes_absrisk_nhc_10y[[endpoint]] <- df[df$set == "test", ]
}

### 1.2.5 Step 2: Absolute Risk 10y NHC

In [ ]:
dir <- "UKBRISK_ENModels/NHC/Secondary_10y"
files <- system(paste0("dx ls ", dir), intern = TRUE)


absrisk_files <- files[grepl("_combined_10y.csv", files)]

for (file_name in absrisk_files) {
  download_cmd <- paste0("dx download ", dir, "/", file_name, " --overwrite")
  system(download_cmd)
}

merged_dataframes <- list()

for (file_name in absrisk_files) {
  
  abs_risk_data <- read.csv(file_name, header = TRUE)
  
  parts <- strsplit(file_name, "_")[[1]]
  
  endpoint <- parts[2] 
  
  risk_combo <- parts[3]
  model_combo <- paste(parts[4:(length(parts) - 3)], collapse = "_")  
  
  combo_name <- paste(risk_combo, model_combo, sep = "_")
  

  if (!is.null(merged_dataframes[[endpoint]])) {
   
    merged_dataframes[[endpoint]] <- merge(
      merged_dataframes[[endpoint]], 
      abs_risk_data[, c("eid", "survival_probability", "set")], 
      by = c("eid", "set"), 
      all = TRUE
    )
    
    colnames(merged_dataframes[[endpoint]])[ncol(merged_dataframes[[endpoint]])] <- combo_name
  } else {
  
    abs_risk_data_merged <- abs_risk_data[, c("eid", "set", "survival_probability")]
    colnames(abs_risk_data_merged)[3] <- combo_name
    merged_dataframes[[endpoint]] <- abs_risk_data_merged
  }
}

cat("All absolute risk data frames merged successfully.\n")

for (endpoint in names(merged_dataframes)) {
  merged_dataframes[[endpoint]] <- merge(
    merged_dataframes[[endpoint]], 
    endpoints_10y[, c("eid", paste0(endpoint, "_followup"), paste0(endpoint, "_status"))], 
    by = "eid", 
    all.x = TRUE
  )
}

train_dataframes_absrisk_nhc_secondary_10y <- list()
test_dataframes_absrisk_nhc_secondary_10y <- list()

for (endpoint in names(merged_dataframes)) {
  df <- merged_dataframes[[endpoint]]
  
  train_dataframes_absrisk_nhc_secondary_10y[[endpoint]] <- df[df$set == "train", ]
  test_dataframes_absrisk_nhc_secondary_10y[[endpoint]] <- df[df$set == "test", ]
}


### 1.2.6 Step 1 + 2: Merged abs risk prediction 10y

In [ ]:
final_abs_nhc_10y_test <- list()

for (endpoint in names(test_dataframes_absrisk_nhc_10y)) {
  
  final_df <- test_dataframes_absrisk_nhc_10y[[endpoint]]

  if (endpoint %in% names(test_dataframes_absrisk_nhc_secondary_10y)) {
  
    secondary_df <- test_dataframes_absrisk_nhc_secondary_10y[[endpoint]]
    
    for (col_name in names(secondary_df)[grepl("^risk_", names(secondary_df))]) {
      
     
      parts <- strsplit(col_name, "_")[[1]]
      
      risk_index <- which(parts == "risk")
      model_index <- which(parts == "model")
      
      risk_combo <- paste(parts[(risk_index + 1):(model_index - 1)], collapse = "_")
      model_combo <- paste(parts[(model_index + 1):(length(parts) - 1)], collapse = "_")
      
      new_col_name <- paste0("initial_", risk_combo, "_secondary_", model_combo)
      
      final_df[[new_col_name]] <- final_df[[risk_combo]]
      
      merged_df <- merge(final_df, secondary_df[, c("eid", col_name)], by = "eid", all.x = TRUE)
      
      final_df[[new_col_name]] <- ifelse(!is.na(merged_df[[col_name]]),
                                         merged_df[[col_name]],
                                         final_df[[new_col_name]])
    }
  }

  final_abs_nhc_10y_test[[endpoint]] <- final_df
}


## 1.3 Predictors

In [ ]:
system("dx download 'UKBRISK_Processed/Processed_final_04092024.tsv' --overwrite")
predictors <- read.delim("Processed_final_04092024.tsv", sep = "\t")

# 2. Benchmarking Results

## 2.1 Read bootstrapped C results

In [ ]:
system("dx download 'UKBRISK/PlotData/absC_benchmarking_bootstrapped.tsv'")
plot_data_abs <- read.delim("absC_benchmarking_bootstrapped.tsv", sep = "\t")

In [ ]:
system("dx download 'UKBRISK/PlotData/deltaC_benchmarking_bootstrapped.tsv'")
plot_data_delta <- read.delim("deltaC_benchmarking_bootstrapped.tsv", sep = "\t")
plot_data_delta <- plot_data_delta %>%
  mutate(across(where(is.numeric), ~ . * -1))

## 2.2 Enhancing clinical risk

### 2.2.1 Data Prep

In [ ]:
# Filter out specific models
plot_data_abs_1 <- plot_data_abs %>% filter(!Model %in% c("pmh", "pmh_ts", "prs_metabolomics_pmh_ts", "clinicalrisk_prs_metabolomics", "score", "qrisk", "prevent"))
plot_data_delta_1 <- plot_data_delta %>% filter(!Model2 %in% c("pmh", "pmh_ts", "prs_metabolomics_pmh_ts", "clinicalrisk_prs_metabolomics", "score", "qrisk", "prevent"))

# Recode the Model column
plot_data_abs_1$Model <- recode(plot_data_abs_1$Model, 
                          "clinicalrisk_pmh_ts" = "Clinical Model + PMH/Questionnaires", 
                          "clinicalrisk" = "Clinical Model", 
                          "everything" = "Clinical Model + PMH/Questionnaires + Omics")
plot_data_delta_1$Model2 <- recode(plot_data_delta_1$Model2, 
                          "clinicalrisk_pmh_ts" = "Clinical Model + PMH/Questionnaires", 
                          "everything" = "Clinical Model + PMH/Questionnaires + Omics")

In [ ]:
# Recode the Endpoints
plot_data_abs_1$Endpoint <- recode(plot_data_abs_1$Endpoint, 
                            "AAA" = "Abdominal aortic aneurysm", 
                            "AD" = "Alzheimer's disease", 
                            "AF" = "Atrial fibrillation", 
                            "AS" = "Asthma",
                            "BC" = "Breast cancer",
                            "CAD" = "Coronary artery disease",
                            "CAT" = "Cataract",
                            "COPD" = "Chronic obstructive pulmonary disease",
                            "CRC" = "Colorectal cancer",
                            "CVD" = "Cardiovascular disease",
                            "DM" = "Diabetes mellitus",
                            "HF" = "Heart failure",
                            "HT" = "Hypertension",
                            "ISS" = "Ischemic stroke",
                            "LC" = "Lung cancer",
                            "LD" = "Liver disease",
                            "MEL" = "Melanoma",
                            "OP" = "Osteoporosis",
                            "PAD" = "Peripheral artery disease",
                            "PC" = "Prostate cancer",
                            "PD" = "Parkinson's disease",
                            "POAG" = "Primary open angle glaucoma",
                            "RD" = "Renal disease",
                            "VT" = "Venous thrombosis")
plot_data_delta_1$Endpoint <- recode(plot_data_delta_1$Endpoint, 
                            "AAA" = "Abdominal aortic aneurysm", 
                            "AD" = "Alzheimer's disease", 
                            "AF" = "Atrial fibrillation", 
                            "AS" = "Asthma",
                            "BC" = "Breast cancer",
                            "CAD" = "Coronary artery disease",
                            "CAT" = "Cataract",
                            "COPD" = "Chronic obstructive pulmonary disease",
                            "CRC" = "Colorectal cancer",
                            "CVD" = "Cardiovascular disease",
                            "DM" = "Diabetes mellitus",
                            "HF" = "Heart failure",
                            "HT" = "Hypertension",
                            "ISS" = "Ischemic stroke",
                            "LC" = "Lung cancer",
                            "LD" = "Liver disease",
                            "MEL" = "Melanoma",
                            "OP" = "Osteoporosis",
                            "PAD" = "Peripheral artery disease",
                            "PC" = "Prostate cancer",
                            "PD" = "Parkinson's disease",
                            "POAG" = "Primary open angle glaucoma",
                            "RD" = "Renal disease",
                            "VT" = "Venous thrombosis")

In [ ]:
# Factor sorting for plotting order
endpoint_order <- plot_data_abs_1 %>%
  group_by(Endpoint) %>%
  summarise(max_cindex = max(C_Index, na.rm = TRUE)) %>%
  arrange(max_cindex) %>%
  pull(Endpoint)

plot_data_abs_1$Endpoint <- factor(plot_data_abs_1$Endpoint, levels = endpoint_order)
plot_data_delta_1$Endpoint <- factor(plot_data_delta_1$Endpoint, levels = endpoint_order)

model_order <- c("Clinical Model", "Clinical Model + PMH/Questionnaires", "Clinical Model + PMH/Questionnaires + Omics")

# Apply the custom order as factor levels based on string matching
plot_data_abs_1$Model <- factor(plot_data_abs_1$Model, levels = model_order)
plot_data_delta_1$Model2 <- factor(plot_data_delta_1$Model2, levels = model_order)



### 2.2.2 Plots

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 13)

#### absolute C

In [ ]:
background_data <- data.frame(
  ymin = seq(0.5, length(levels(as.factor(plot_data_abs_1$Endpoint))) - 0.5, 2),
  ymax = seq(1.5, length(levels(as.factor(plot_data_abs_1$Endpoint))) + 0.5, 2),
  xmin = -Inf,
  xmax = Inf
)

abs_plot_1 <- ggplot(plot_data_abs_1, aes(x = C_Index, y = Endpoint, fill = Model, color = Model)) +
    geom_rect(data = background_data, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
            fill = "lightgray", alpha = 0.5, inherit.aes = FALSE) +
    geom_errorbarh(aes(xmin = Bottom95, xmax = Top95), height = 0.2, position = position_dodge(width = -0.5)) +
    geom_point(size = 5, shape = 21, stroke = 0.6, position = position_dodge(width = -0.5)) + 
    scale_fill_manual(values = c("Clinical Model" = "#FF9966",
                               "Clinical Model + PMH/Questionnaires" = "#D7BDE2",
                               "Clinical Model + Omics" = "#FADBD8",
                               "Clinical Model + PMH/Questionnaires + Omics" = "#9B59B6")) +
    scale_color_manual(values = c("Clinical Model" = "black",
                               "Clinical Model + PMH/Questionnaires" = "black",
                               "Clinical Model + Omics" = "black",
                               "Clinical Model + PMH/Questionnaires + Omics" = "black")) +
    theme_minimal() +
    labs(x = "Harrell's C", y = "") +
    theme(
        legend.position = "bottom",
        legend.title = element_text(size = 12),
        legend.text = element_text(size = 12),
        axis.line = element_line(size = 1),  
        axis.ticks = element_line(size = 1),  
        axis.text = element_text(size = 14),  
        axis.title.x = element_text(size = 14),  
        axis.title.y = element_text(size = 14)  
      )

abs_plot_1

#### delta C

In [ ]:
delta_plot_1 <- ggplot(plot_data_delta_1, aes(x = MeanDeltaC, y = Endpoint, fill = Model2)) +
    geom_rect(data = background_data, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
            fill = "lightgray", alpha = 0.5, inherit.aes = FALSE) +
    geom_errorbarh(aes(xmin = Bottom95, xmax = Top95), height = 0.2, position = position_dodge(width = -0.5)) +
    geom_vline(xintercept = 0, linetype = "dashed", color = "#FF9966", size = 1.2) +  
    geom_point(aes(fill = Model2), color = "black", size = 5, shape = 21, stroke = 0.6, position = position_dodge(width = -0.5)) +
    scale_fill_manual(values = c("Clinical Model" = "#FF9966",
                               "Clinical Model + PMH/Questionnaires" = "#D7BDE2",
                               "Clinical Model + Omics" = "#FADBD8",
                               "Clinical Model + PMH/Questionnaires + Omics" = "#9B59B6")) +
  scale_color_manual(values = c("Clinical Model" = "black",
                               "Clinical Model + PMH/Questionnaires" = "black",
                               "Clinical Model + Omics" = "black",
                               "Clinical Model + PMH/Questionnaires + Omics" = "black")) +
    theme_minimal() +
    labs(x = "ΔHarrell's C (Baseline: Clinical Model)", y = "") +
    theme(
    legend.position = "bottom",
    legend.title = element_text(size = 12),
    legend.text = element_text(size = 12),
    axis.line = element_line(size = 1),  
    axis.ticks = element_line(size = 1),  
    axis.text = element_text(size = 14),  
    axis.title.x = element_text(size = 14),  
    axis.title.y = element_text(size = 14)  
  )

delta_plot_1


## 2.3 clinical risk vs cost optimised

### 2.3.1 Data Prep

In [ ]:
# Filter out specific models
plot_data_abs_2 <- plot_data_abs %>% filter(!Model %in% c("pmh", "clinicalrisk_pmh_ts", "clinicalrisk_prs_metabolomics", "everything", "score", "qrisk", "prevent"))
plot_data_delta_2 <- plot_data_delta %>% filter(!Model2 %in% c("pmh", "clinicalrisk_pmh_ts", "clinicalrisk_prs_metabolomics", "everything", "score", "qrisk", "prevent"))

# Recode the Model column
plot_data_abs_2$Model <- recode(plot_data_abs_2$Model, 
                          "pmh_ts" = "No Needle Model", 
                          "prs_metabolomics_pmh_ts" = "Single Blood Draw Model", 
                          "clinicalrisk" = "Clinical Model")
plot_data_delta_2$Model2 <- recode(plot_data_delta_2$Model2, 
                          "pmh_ts" = "No Needle Model", 
                          "prs_metabolomics_pmh_ts" = "Single Blood Draw Model")

In [ ]:
# Recode the Endpoints
plot_data_abs_2$Endpoint <- recode(plot_data_abs_2$Endpoint, 
                            "AAA" = "Abdominal aortic aneurysm", 
                            "AD" = "Alzheimer's disease", 
                            "AF" = "Atrial fibrillation", 
                            "AS" = "Asthma",
                            "BC" = "Breast cancer",
                            "CAD" = "Coronary artery disease",
                            "CAT" = "Cataract",
                            "COPD" = "Chronic obstructive pulmonary disease",
                            "CRC" = "Colorectal cancer",
                            "CVD" = "Cardiovascular disease",
                            "DM" = "Diabetes mellitus",
                            "HF" = "Heart failure",
                            "HT" = "Hypertension",
                            "ISS" = "Ischemic stroke",
                            "LC" = "Lung cancer",
                            "LD" = "Liver disease",
                            "MEL" = "Melanoma",
                            "OP" = "Osteoporosis",
                            "PAD" = "Peripheral artery disease",
                            "PC" = "Prostate cancer",
                            "PD" = "Parkinson's disease",
                            "POAG" = "Primary open angle glaucoma",
                            "RD" = "Renal disease",
                            "VT" = "Venous thrombosis")
plot_data_delta_2$Endpoint <- recode(plot_data_delta_2$Endpoint, 
                            "AAA" = "Abdominal aortic aneurysm", 
                            "AD" = "Alzheimer's disease", 
                            "AF" = "Atrial fibrillation", 
                            "AS" = "Asthma",
                            "BC" = "Breast cancer",
                            "CAD" = "Coronary artery disease",
                            "CAT" = "Cataract",
                            "COPD" = "Chronic obstructive pulmonary disease",
                            "CRC" = "Colorectal cancer",
                            "CVD" = "Cardiovascular disease",
                            "DM" = "Diabetes mellitus",
                            "HF" = "Heart failure",
                            "HT" = "Hypertension",
                            "ISS" = "Ischemic stroke",
                            "LC" = "Lung cancer",
                            "LD" = "Liver disease",
                            "MEL" = "Melanoma",
                            "OP" = "Osteoporosis",
                            "PAD" = "Peripheral artery disease",
                            "PC" = "Prostate cancer",
                            "PD" = "Parkinson's disease",
                            "POAG" = "Primary open angle glaucoma",
                            "RD" = "Renal disease",
                            "VT" = "Venous thrombosis")

In [ ]:
# Factor sorting for plotting order
# endpoint order defined above

plot_data_abs_2$Endpoint <- factor(plot_data_abs_2$Endpoint, levels = endpoint_order)
plot_data_delta_2$Endpoint <- factor(plot_data_delta_2$Endpoint, levels = endpoint_order)

model_order <- c("No Needle Model",
                 "Single Blood Draw Model",
                 "Clinical Model")

# Apply the custom order as factor levels based on string matching
plot_data_abs_2$Model <- factor(plot_data_abs_2$Model, levels = model_order)
plot_data_delta_2$Model2 <- factor(plot_data_delta_2$Model2, levels = model_order)

### 2.3.2 Plots

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 13)

#### abs C

In [ ]:
abs_plot_2 <- ggplot(plot_data_abs_2, aes(x = C_Index, y = Endpoint, fill = Model, color = Model)) +
    geom_rect(data = background_data, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
            fill = "lightgray", alpha = 0.5, inherit.aes = FALSE) +
    geom_errorbarh(aes(xmin = Bottom95, xmax = Top95), height = 0.2, position = position_dodge(width = 0.5)) +
    geom_point(size = 5, shape = 21, stroke = 0.6, position = position_dodge(width = 0.5)) + 
    scale_fill_manual(values = c("No Needle Model" = "#AED6F1",
                               "Single Blood Draw Model" = "#2E86C1",
                               "Clinical Model" = "#FF9966")) +
    scale_color_manual(values = c("No Needle Model" = "black",
                               "Single Blood Draw Model" = "black",
                               "Clinical Model" = "black")) +
    theme_minimal() +
    labs(x = "Harrell's C", y = "") +
    theme(
        legend.position = "bottom",
        legend.title = element_text(size = 12),
        legend.text = element_text(size = 12),
        axis.line = element_line(size = 1),  
        axis.ticks = element_line(size = 1),  
        axis.text = element_text(size = 14),  
        axis.title.x = element_text(size = 14),  
        axis.title.y = element_text(size = 14)  
      )

abs_plot_2

#### delta C

In [ ]:
delta_plot_2 <- ggplot(plot_data_delta_2, aes(x = MeanDeltaC, y = Endpoint, fill = Model2)) +
    geom_rect(data = background_data, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
            fill = "lightgray", alpha = 0.5, inherit.aes = FALSE) +
    geom_errorbarh(aes(xmin = Bottom95, xmax = Top95), height = 0.2, position = position_dodge(width = 0.5)) +
    geom_vline(xintercept = 0, linetype = "dashed", color = "#FF9966", size = 1.2) + 
    geom_point(aes(fill = Model2), color = "black", size = 5, shape = 21, stroke = 0.6, position = position_dodge(width = 0.5)) +
    scale_fill_manual(values = c("No Needle Model" = "#AED6F1",
                               "Single Blood Draw Model" = "#2E86C1",
                               "Clinical Model" = "#FF9966")) +
    scale_color_manual(values = c("No Needle Model" = "black",
                               "Single Blood Draw Model" = "black",
                               "Clinical Model" = "black")) +
    theme_minimal() +
    labs(x = "ΔHarrell's C (Baseline: Clinical Model)", y = "") +
    theme(
    legend.position = "bottom",
    legend.title = element_text(size = 12),
    legend.text = element_text(size = 12),
    axis.line = element_line(size = 1),  
    axis.ticks = element_line(size = 1),  
    axis.text = element_text(size = 14),  
    axis.title.x = element_text(size = 14),  
    axis.title.y = element_text(size = 14)  
  )

delta_plot_2


## 2.4 Survival plots

In [ ]:
endpoint_labels <- c(
  "AAA" = "Abdominal aortic aneurysm", 
  "AD" = "Alzheimer's disease", 
  "AF" = "Atrial fibrillation", 
  "AS" = "Asthma",
  "BC" = "Breast cancer",
  "CAD" = "Coronary artery disease",
  "CAT" = "Cataract",
  "COPD" = "Chronic obstructive pulmonary disease",
  "CRC" = "Colorectal cancer",
  "CVD" = "Cardiovascular disease",
  "DM" = "Diabetes mellitus",
  "HF" = "Heart failure",
  "HT" = "Hypertension",
  "ISS" = "Ischemic stroke",
  "LC" = "Lung cancer",
  "LD" = "Liver disease",
  "MEL" = "Melanoma",
  "OP" = "Osteoporosis",
  "PAD" = "Peripheral artery disease",
  "PC" = "Prostate cancer",
  "PD" = "Parkinson's disease",
  "POAG" = "Primary open angle glaucoma",
  "RD" = "Renal disease",
  "VT" = "Venous thrombosis"
)

In [ ]:
survival_plots <- list()

palette <- c("#b9b0b0", "#3e3636", "999090")

for (endpoint in names(merged_test_dataframes)) {
  
  df <- merged_test_dataframes[[endpoint]]
  
  df$pmh_ts_tertiles <- cut(df$pmh_ts, breaks = quantile(df$pmh_ts, probs = seq(0, 1, 1/3), na.rm = TRUE), include.lowest = TRUE)
  
  surv_object <- Surv(time = df[[paste0(endpoint, "_followup")]], event = df[[paste0(endpoint, "_status")]])

 
  fit <- survfit(surv_object ~ pmh_ts_tertiles, data = df)

  survival_plot <- ggsurvplot(fit, data = df, pval = FALSE, risk.table = FALSE, 
                              fun = "event", 
                              conf.int = TRUE, 
                              title = endpoint_labels[endpoint],  
                              palette = palette,
                              censor = FALSE, 
                              ggtheme = theme_minimal()) 
  
  
  survival_plot$plot <- survival_plot$plot +
    theme(
        legend.position = "none", 
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(), 
        axis.line = element_line(size = 1), 
        axis.ticks = element_line(size = 1),
        axis.title = element_blank(), 
        axis.title.y = element_blank(), 
        plot.title = element_text(size = 12, hjust = 0.5), 
        plot.margin = unit(c(0.5, 0, 0, 0), "cm")
    ) +
    
    scale_y_continuous(labels = scales::percent) +
    scale_x_continuous(breaks = c(0, 5, 10, 15))
  
 
  survival_plots[[endpoint]] <- survival_plot
}

## 2.5 Finalize Fig 1

##### top panel

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

In [ ]:
empty_plot <- ggplot() + theme_void() + theme(plot.margin = unit(c(0.5, 0.5, 0.5, 0.5), "cm"))

remove_legend_and_axis_text <- function(p) {
    return(p + theme(
      legend.position = "none",               
      axis.text.y = element_blank(),          
      axis.title.y = element_blank(),          
      plot.margin = unit(c(0.5, 0.4, 0, 0), "cm")  
    ))
}

abs_plot_1_left_axis <- remove_legend_and_axis_text(abs_plot_1)  
delta_plot_1_no_axis <- remove_legend_and_axis_text(delta_plot_1)
abs_plot_2_no_axis <- remove_legend_and_axis_text(abs_plot_2)
delta_plot_2_no_axis <- remove_legend_and_axis_text(delta_plot_2)


top_plots_with_left_y_axis <- plot_grid(
  empty_plot, abs_plot_1_left_axis, delta_plot_1_no_axis, abs_plot_2_no_axis, delta_plot_2_no_axis,
  ncol = 5, align = "v", rel_widths = c(0.7, 1, 1, 1, 1)  
)


In [ ]:
top_plots_with_left_y_axis

In [ ]:
ggsave(
    filename = "fig1_a.svg",   
    plot = top_plots_with_left_y_axis,                   
    width = 20, height = 10,                        
    dpi = 1000,
    bg = "transparent" 
)

uplcmd <- "dx upload fig1_a.svg --path UKBRISK/Plots/fig1_a.svg"
system(uplcmd)

In [ ]:
ggsave(
    filename = "fig1_a_labels.svg",   
    plot = abs_plot_1,                   
    width = 20, height = 10,                        
    dpi = 1000,
    bg = "transparent" 
)

uplcmd <- "dx upload fig1_a_labels.svg --path UKBRISK/Plots/fig1_a_labels.svg"
system(uplcmd)

In [ ]:
ggsave(
    filename = "fig1_a_legend.svg",   
    plot = abs_plot_2,                   
    width = 20, height = 10,                       
    dpi = 1000,
    bg = "transparent" 
)

uplcmd <- "dx upload fig1_a_legend.svg --path UKBRISK/Plots/fig1_a_legend.svg"
system(uplcmd)

##### bottom grid

In [ ]:
nrow <- 4
ncol <- 6

for (i in seq_along(survival_plots)) {
  row_position <- ceiling(i / ncol)
  is_bottom_row <- (row_position == nrow)
  
  if (!is_bottom_row) {
    survival_plots[[i]]$plot <- survival_plots[[i]]$plot +
      theme(axis.text.x = element_blank())  
  }
}

plot_list <- lapply(survival_plots, function(x) x$plot)

aligned_plots <- plot_grid(plotlist = plot_list, nrow = nrow, ncol = ncol, align = "hv")


In [ ]:
aligned_plots

In [ ]:
survival_1 <- ggdraw() +
  draw_plot(aligned_plots, 0.03, 0.08, 0.9, 0.9) +  
  draw_label("Follow-up Time (Years)", x = 0.5, y = 0.05, vjust = 0, size = 14) +  
  draw_label("Cumulative Events (%)", x = 0.02, y = 0.5, angle = 90, vjust = 0, size = 14) 

survival_1

In [ ]:
ggsave(
    filename = "fig1_b.svg",   
    plot = survival_1,                    
    width = 20, height = 10,                       
    dpi = 1000,
    bg = "transparent" 
)

uplcmd <- "dx upload fig1_b.svg --path UKBRISK/Plots/fig1_b.svg"
system(uplcmd)

# 3. CV risk stratification

## 3.1 Forrest Plots

### 3.1.1 Read bootstrapped C

In [ ]:
system("dx download 'UKBRISK/PlotData/absC_benchmarking_bootstrapped_cvd.tsv'")
plot_data_abs_cvd <- read.delim("absC_benchmarking_bootstrapped_cvd.tsv", sep = "\t")

In [ ]:
system("dx download 'UKBRISK/PlotData/deltaC_benchmarking_bootstrapped_cvd.tsv'")
plot_data_delta_cvd <- read.delim("deltaC_benchmarking_bootstrapped_cvd.tsv", sep = "\t")
plot_data_delta_cvd <- plot_data_delta_cvd %>%
  mutate(across(where(is.numeric), ~ . * -1))

In [ ]:
levels(as.factor(plot_data_abs_cvd$Model))

### 3.1.2 Data Prep

In [ ]:
plot_data_abs_cvd$Model <- recode(plot_data_abs_cvd$Model, 
                                    "pmh_ts" = "No Needle Model", 
                                    "prevent" = "PREVENT", 
                                    "prs_metabolomics_pmh_ts" = "Single Blood Draw Model",
                                    "qrisk" = "QRISK3",
                                    "score" = "SCORE2"
                                 )
plot_data_delta_cvd$Model2 <- recode(plot_data_delta_cvd$Model2, 
                                    "pmh_ts" = "No Needle Model", 
                                    "prevent" = "PREVENT", 
                                    "prs_metabolomics_pmh_ts" = "Single Blood Draw Model",
                                    "qrisk" = "QRISK3",
                                    "score" = "SCORE2"
                                 )

In [ ]:
plot_data_abs_cvd$Endpoint <- recode(plot_data_abs_cvd$Endpoint, 
                            "CAD" = "Coronary artery disease",
                            "CVD" = "Cardiovascular disease",
                            "HF" = "Heart failure",
                            "ISS" = "Ischemic stroke",
                            "PAD" = "Peripheral artery disease")
plot_data_delta_cvd$Endpoint <- recode(plot_data_delta_cvd$Endpoint, 
                            "CAD" = "Coronary artery disease",
                            "CVD" = "Cardiovascular disease",
                            "HF" = "Heart failure",
                            "ISS" = "Ischemic stroke",
                            "PAD" = "Peripheral artery disease")

In [ ]:
endpoint_order <- plot_data_abs_cvd %>%
  group_by(Endpoint) %>%
  summarise(max_cindex = max(C_Index, na.rm = TRUE)) %>%
  arrange(max_cindex) %>%
  pull(Endpoint)

plot_data_abs_cvd$Endpoint <- factor(plot_data_abs_cvd$Endpoint, levels = endpoint_order)
plot_data_delta_cvd$Endpoint <- factor(plot_data_delta_cvd$Endpoint, levels = endpoint_order)

model_order <- c("SCORE2", "PREVENT", "QRISK3", "No Needle Model", "Single Blood Draw Model")

plot_data_abs_cvd$Model <- factor(plot_data_abs_cvd$Model, levels = model_order)
plot_data_delta_cvd$Model2 <- factor(plot_data_delta_cvd$Model2, levels = model_order)



### 3.1.3 Absolute C Plot

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

In [ ]:
background_data <- data.frame(
  ymin = seq(0.5, length(levels(as.factor(plot_data_abs_cvd$Endpoint))) - 0.5, 2),
  ymax = seq(1.5, length(levels(as.factor(plot_data_abs_cvd$Endpoint))) + 0.5, 2),
  xmin = -Inf,
  xmax = Inf
)

abs_plot_cvd <- ggplot(plot_data_abs_cvd, aes(x = C_Index, y = Endpoint, fill = Model, color = Model)) +
    geom_rect(data = background_data, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
            fill = "lightgray", alpha = 0.5, inherit.aes = FALSE) +
    geom_errorbarh(aes(xmin = Bottom95, xmax = Top95), height = 0.2, position = position_dodge(width = -0.5)) +
    geom_point(size = 5, shape = 21, stroke = 0.6, position = position_dodge(width = -0.5)) + 
    scale_fill_manual(values = c("No Needle Model" = "#AED6F1",
                               "Single Blood Draw Model" = "#2E86C1",
                               "SCORE2" = "#F69697",
                               "PREVENT" = "#FF2C2C",
                               "QRISK3" = "#C30010")) +
    scale_color_manual(values = c("No Needle Model" = "black",
                               "Single Blood Draw Model" = "black",
                               "SCORE2" = "black",
                               "PREVENT" = "black",
                               "QRISK3" = "black")) +
    theme_minimal() +
    labs(x = "Harrell's C", y = "") +
    theme(
        legend.position = "bottom",
        legend.title = element_text(size = 12),
        legend.text = element_text(size = 12),
        axis.line = element_line(size = 1),  
        axis.ticks = element_line(size = 1),  
        axis.text = element_text(size = 14),  
        axis.title.x = element_text(size = 14),  
        axis.title.y = element_text(size = 14)  
      )

abs_plot_cvd

### 3.1.4 Relative C Plot

In [ ]:
delta_plot_cvd <- ggplot(plot_data_delta_cvd, aes(x = MeanDeltaC, y = Endpoint, fill = Model2)) +
    geom_rect(data = background_data, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
            fill = "lightgray", alpha = 0.5, inherit.aes = FALSE) +
    geom_errorbarh(aes(xmin = Bottom95, xmax = Top95), height = 0.2, position = position_dodge(width = -0.5)) +
    geom_vline(xintercept = 0, linetype = "dashed", color = "#C30010", size = 1.2) +  
    geom_point(aes(fill = Model2), color = "black", size = 5, shape = 21, stroke = 0.6, position = position_dodge(width = -0.5)) +
    scale_fill_manual(values = c("No Needle Model" = "#AED6F1",
                               "Single Blood Draw Model" = "#2E86C1",
                               "SCORE2" = "#F69697",
                               "PREVENT" = "#FF2C2C",
                               "QRISK3" = "#C30010")) +
    scale_color_manual(values = c("No Needle Model" = "black",
                               "Single Blood Draw Model" = "black",
                               "SCORE2" = "black",
                               "PREVENT" = "black",
                               "QRISK3" = "black")) +
    theme_minimal() +
    labs(x = "ΔHarrell's C (Baseline: QRISK3)", y = "") +
    theme(
    legend.position = "bottom",
    legend.title = element_text(size = 12),
    legend.text = element_text(size = 12),
    axis.line = element_line(size = 1),  
    axis.ticks = element_line(size = 1),  
    axis.text = element_text(size = 14),  
    axis.title.x = element_text(size = 14),  
    axis.title.y = element_text(size = 14)  
  )

delta_plot_cvd


## 3.2 ROC Plots

In [ ]:
# these are also used in DCA plots
predictors <- c("pmh_ts" = "No Needle Model", 
                "prevent" = "PREVENT", 
                "prs_metabolomics_pmh_ts" = "Single Blood Draw Model",
                "qrisk" = "QRISK3",
                "score" = "SCORE2")

endpoints <- c("CAD" = "Coronary artery disease",
               "CVD" = "Cardiovascular disease",
               "HF" = "Heart failure",
               "ISS" = "Ischemic stroke",
               "PAD" = "Peripheral artery disease")

custom_colors <- c("No Needle Model" = "#AED6F1",
                   "Single Blood Draw Model" = "#2E86C1",
                   "SCORE2" = "#F69697",
                   "PREVENT" = "#FF2C2C",
                   "QRISK3" = "#C30010")

In [ ]:
roc_plots <- list()

for (endpoint in names(endpoints)) {
 
  df <- merged_test_dataframes[[endpoint]]
  
  status_col <- paste0(endpoint, "_status")
  if (!status_col %in% colnames(df)) {
    cat("Skipping endpoint", endpoint, "- missing status column.\n")
    next
  }
  
  roc_data_list <- list()
  
  for (predictor in names(predictors)) {
    if (predictor %in% colnames(df)) {
      roc_curve <- roc(df[[status_col]], df[[predictor]], plot = FALSE)
      
      roc_df <- data.frame(
        Specificity = 1 - roc_curve$specificities,
        Sensitivity = roc_curve$sensitivities,
        Model = predictors[predictor]
      )
      
      roc_data_list[[predictor]] <- roc_df
    } else {
      cat("Predictor", predictor, "not found in the dataframe for", endpoint, ".\n")
    }
  }

  combined_roc_df <- do.call(rbind, roc_data_list)
  
  endpoint_plot <- ggplot(combined_roc_df, aes(x = Specificity, y = Sensitivity, color = Model)) +
    geom_line(size = 1) +
    geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "gray", size = 1) +
    labs(x = "1 - Specificity", y = "Sensitivity", color = "Model") +
    ggtitle(endpoints[endpoint]) +
    scale_color_manual(values = custom_colors) +
    theme_minimal() +
    theme(
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(), 
        legend.position = "bottom",
        legend.title = element_text(size = 12),
        legend.text = element_text(size = 12),
        axis.line = element_line(size = 1),  
        axis.ticks = element_line(size = 1), 
        axis.title.x = element_blank(),  
        axis.title.y = element_blank(),
        plot.title = element_text(size = 14, hjust = 0.5)
  )
  
  roc_plots[[endpoint]] <- endpoint_plot
}

## 3.3 DCA Plots

In [ ]:
install.packages("dcurves")
library(dcurves)

In [ ]:
dca_data <- list()

for (endpoint in names(test_dataframes_absrisk_cv_10y)) {

  df <- test_dataframes_absrisk_cv_10y[[endpoint]]
  
  status_col <- paste0(endpoint, "_status")
  
  dca_results <- list()


  for (predictor in names(predictors)) {
      
    df[[predictor]] <- 1 - df[[predictor]]


    dca_curve <- dca(
      formula = as.formula(paste(status_col, "~", predictor)),
      data = df,
      thresholds = seq(0.01, 0.3, by = 0.01)
    )
    

    net_benefit_values <- dca_curve$dca$net_benefit
    

    treat_all <- net_benefit_values[1:30]   # First 30 are "Treat All"
    treat_none <- net_benefit_values[31:60] # Next 30 are "Treat None"
    actual_model_nb <- net_benefit_values[61:90]  # Last 30 are for the actual model


    actual_model_df <- data.frame(
      threshold = dca_curve$dca$threshold,
      net_benefit = actual_model_nb,
      label = predictors[predictor],
      curve_type = "Model"
    )
    
    # Create dataframes for "Treat All" and "Treat None"
    treat_all_df <- data.frame(
      threshold = dca_curve$dca$threshold,
      net_benefit = treat_all,
      label = predictors[predictor],
      curve_type = "Treat All"
    )
    
    treat_none_df <- data.frame(
      threshold = dca_curve$dca$threshold,
      net_benefit = treat_none,
      label = predictors[predictor],
      curve_type = "Treat None"
    )
    
    # Combine the results for this predictor and store them
    combined_df <- rbind(actual_model_df, treat_all_df, treat_none_df)
    dca_results[[predictor]] <- combined_df
  }
  
  # Combine all results into a single dataframe for this endpoint
  if (length(dca_results) > 0) {
    combined_dca_df <- do.call(rbind, dca_results)
    dca_data[[endpoint]] <- combined_dca_df
  }
}

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 3)

In [ ]:
all_plots <- list()


for (endpoint in names(endpoints)) {
  
  
  treat_all_curve  <- subset(dca_data[[endpoint]],  curve_type == "Treat All")
    treat_none_curve <- subset(dca_data[[endpoint]],  curve_type == "Treat None")
  
 
  y_lim_max <- max(treat_all_curve$net_benefit)
    y_lim_min <- -(max(treat_all_curve$net_benefit)/10)
  

  p <- ggplot() +
    
    lapply(names(predictors), function(predictor) {
      model_curve <- subset(dca_data[[endpoint]], label == predictors[predictor] & curve_type == "Model")
      geom_line(data = model_curve, aes(x = threshold, y = net_benefit, color = predictors[predictor]), size = 1)
    }) +
    

    geom_line(data = treat_all_curve, aes(x = threshold, y = net_benefit), color = "darkred", size = 1, linetype = "dashed") +
    

    geom_line(data = treat_none_curve, aes(x = threshold, y = net_benefit), color = "darkgrey", size = 1, linetype = "solid") +
    

    scale_color_manual(values = custom_colors) +  
    labs(title = endpoints[endpoint]) +
    theme_minimal() +
    theme(
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(),
      axis.line = element_line(size = 1),
      axis.ticks = element_line(size = 1),
      axis.title.x = element_blank(),  
      axis.title.y = element_blank(),  
      plot.title = element_text(hjust = 0.5, size = 14),  
      plot.margin = margin(t = 5, r = 5, b = 5, l = 5), 
      legend.position = "none"  
    ) +
    coord_cartesian(ylim = c(y_lim_min, y_lim_max))  
  
  all_plots[[endpoint]] <- p
}


## 3.4 Finalize fig 3

In [ ]:

empty_plot <- ggplot() + theme_void() + theme(plot.margin = unit(c(0.5, 0.5, 0.5, 0.5), "cm"))


remove_legend_and_axis_text <- function(p) {
    return(p + theme(
      legend.position = "none",               
      axis.text.y = element_blank(),           
      axis.title.y = element_blank(),         
      plot.margin = unit(c(0.5, 0.4, 0, 0), "cm")  
    ))
}


abs_plot_cvd_left_axis <- remove_legend_and_axis_text(abs_plot_cvd)  
delta_plot_cvd_no_axis <- remove_legend_and_axis_text(delta_plot_cvd)


top_plots_fig3 <- plot_grid(
  empty_plot, abs_plot_cvd_left_axis, delta_plot_cvd_no_axis,
  ncol = 3, align = "v", rel_widths = c(0.5, 1, 1)  
)

top_plots_fig3

In [ ]:
ggsave(
  filename = "fig3a.svg",
    plot = top_plots_fig3, 
  width = 20, height = 8,   
  dpi = 1000,                
  bg = "transparent"        
)

system("dx upload fig3a.svg --path UKBRISK/Plots/fig3a.svg")


In [ ]:
ggsave(
  filename = "fig3a_labels.svg",
    plot = abs_plot_cvd, 
  width = 20, height = 8,  
  dpi = 1000,               
  bg = "transparent"        
)

system("dx upload fig3a_labels.svg --path UKBRISK/Plots/fig3a_labels.svg")


In [ ]:
ordered_endpoints <- c("HF", "CAD", "PAD", "ISS", "CVD")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 4)

In [ ]:
for (i in seq_along(ordered_endpoints)) {
  plot_name <- ordered_endpoints[i]
  

  roc_plots[[plot_name]] <- roc_plots[[plot_name]] +
    theme(legend.position = "none", axis.title = element_blank()) 
  

  if (i != 1) {
    roc_plots[[plot_name]] <- roc_plots[[plot_name]] + theme(axis.text.y = element_blank(), axis.ticks.y = element_blank())
  }
}


final_plot_roc <- plot_grid(plotlist = lapply(ordered_endpoints, function(x) roc_plots[[x]]), align = "hv", ncol = length(roc_plots))


final_plot_with_labels_roc <- ggdraw() +
  draw_plot(final_plot_roc, 0, 0, 1, 1)

In [ ]:
final_plot_roc <- ggdraw() +
  draw_plot(final_plot_with_labels_roc, 0.025, 0.08, 0.9, 0.9) +  
  draw_label("1 - Specificity", x = 0.5, y = 0.05, vjust = 0, size = 14) +  
  draw_label("Sensitivity", x = 0.02, y = 0.5, angle = 90, vjust = 0, size = 14)  


In [ ]:
print(final_plot_roc)

In [ ]:
ggsave(
  filename = "fig3b.png",
    plot = final_plot_roc, 
  width = 20, height = 4,   
  dpi = 1000,                
  bg = "transparent"        
)

system("dx upload fig3b.png --path UKBRISK/Plots/fig3b.png")

In [ ]:
final_plot_dca <- plot_grid(plotlist = lapply(ordered_endpoints, function(x) all_plots[[x]]), align = "hv", ncol = length(all_plots))

final_plot_with_labels_dca <- ggdraw() +
  draw_plot(final_plot_dca, 0, 0, 1, 1)

In [ ]:
final_plot_dca <- ggdraw() +
  draw_plot(final_plot_with_labels_dca, 0.025, 0.08, 0.9, 0.9) +  
  draw_label("Treshold Probability", x = 0.5, y = 0.05, vjust = 0, size = 14) +  
  draw_label("Net Benefit", x = 0.02, y = 0.5, angle = 90, vjust = 0, size = 14)  


print(final_plot_dca)

In [ ]:
ggsave(
  filename = "fig3c.png",
    plot = final_plot_dca,
  width = 20, height = 4,
    dpi = 1000,
  bg = "transparent"       
)

system("dx upload fig3c.png --path UKBRISK/Plots/fig3c.png")

# 4. Feature Importance

## 4.1 Data Prep

In [ ]:
library(dplyr)
summary_df <- data.frame(endpoint = character(), category = character(), percentage = numeric(), stringsAsFactors = FALSE)


for (endpoint in names(merged_coef_dataframes)) {

  df <- merged_coef_dataframes[[endpoint]]
  
 
  df_summary <- df %>%
    group_by(Category) %>%
    summarise(total_abs_coef = sum(abs(everything), na.rm = TRUE)) %>%
    
    filter(!Category %in% c("prevent", "qrisk", "score"))
  
 
  total_abs_coef_endpoint <- sum(df_summary$total_abs_coef)
  
 
  df_summary <- df_summary %>%
    mutate(percentage = (total_abs_coef / total_abs_coef_endpoint) * 100) %>%
    mutate(endpoint = endpoint)
  

  df_summary <- df_summary %>%
    select(endpoint, category = Category, percentage)
  
 
  summary_df <- bind_rows(summary_df, df_summary)
}


print(summary_df)


In [ ]:
summary_df2 <- data.frame(endpoint = character(), ts_cat = character(), percentage = numeric(), stringsAsFactors = FALSE)

for (endpoint in names(merged_coef_dataframes)) {

  df <- merged_coef_dataframes[[endpoint]]
  
  df <- df %>%
    filter(Category == "ts")
  
  df_summary <- df %>%
    group_by(ts_cat) %>%
    summarise(total_abs_coef = sum(abs(everything), na.rm = TRUE))
  
  total_abs_coef_endpoint <- sum(df_summary$total_abs_coef)
  
  df_summary <- df_summary %>%
    mutate(percentage = (total_abs_coef / total_abs_coef_endpoint) * 100) %>%
    mutate(endpoint = endpoint)
  
  df_summary <- df_summary %>%
    select(endpoint, ts_cat, percentage)
    
  summary_df2 <- bind_rows(summary_df2, df_summary)    
    summary_df2 <- summary_df2[!is.na(summary_df2$ts_cat), ]
}


In [ ]:
category_names <- c(
  "agesex" = "Age + Sex",
  "ts" = "Questionnaire",
  "prs" = "Genomics",
  "pmh" = "Past Medical History",
  "metabolomics" = "Metabolomics",
  "clinicalrisk" = "Clinical Score"
)

In [ ]:
endpoint_names <- c("AAA" = "Abdominal aortic aneurysm", 
                    "AD" = "Alzheimer's disease", 
                    "AF" = "Atrial fibrillation", 
                    "AS" = "Asthma",
                    "BC" = "Breast cancer",
                    "CAD" = "Coronary artery disease",
                    "CAT" = "Cataract",
                    "COPD" = "Chronic obstructive pulmonary disease",
                    "CRC" = "Colorectal cancer",
                    "CVD" = "Cardiovascular disease",
                    "DM" = "Diabetes mellitus",
                    "HF" = "Heart failure",
                    "HT" = "Hypertension",
                    "ISS" = "Ischemic stroke",
                    "LC" = "Lung cancer",
                    "LD" = "Liver disease",
                    "MEL" = "Melanoma",
                    "OP" = "Osteoporosis",
                    "PAD" = "Peripheral artery disease",
                    "PC" = "Prostate cancer",
                    "PD" = "Parkinson's disease",
                    "POAG" = "Primary open angle glaucoma",
                    "RD" = "Renal disease",
                    "VT" = "Venous thrombosis")

## 4.2 Radar Plot

In [ ]:
install.packages("fmsb")
library(fmsb)

In [ ]:
library(dplyr)
library(fmsb)

big_endpoint <- "CVD"  # <- change

cat_code_order <- names(category_names)
endpoints <- unique(summary_df$endpoint)
endpoints_plot <- c(big_endpoint, setdiff(endpoints, big_endpoint))

wrap_label <- function(x) {
  x <- gsub("Past Medical History", "Past Medical\nHistory", x, fixed = TRUE)
  x <- gsub("Clinical Score", "Clinical Risk Factors", x, fixed = TRUE)
  x <- gsub("Age + Sex", "Age + Sex", x, fixed = TRUE)
  x
}

draw_radar <- function(endpoint, show_labels = FALSE) {
  endpoint_title <- recode(endpoint, !!!endpoint_names)

  df_endpoint <- summary_df %>%
    filter(endpoint == !!endpoint) %>%
    mutate(
      category_code = category,
      category = recode(category, !!!category_names)
    ) %>%
    arrange(match(category_code, cat_code_order))

  vals <- df_endpoint$percentage
  labs <- df_endpoint$category

  df_radar <- as.data.frame(rbind(
    rep(100, length(vals)),
    rep(0,   length(vals)),
    vals
  ))
  colnames(df_radar) <- if (show_labels) wrap_label(labs) else labs

  par(xpd = NA)

  if (show_labels) {
    par(mai = c(1.00, 1.05, 1.10, 1.00))

    radarchart(
      df_radar,
      axistype = 1,
      pcol = "darkred",
      pfcol = rgb(1, 0, 0, 0.30),
      plwd = 3,
      cglcol = "darkgrey",
      cglty = 3,
      cglwd = 1.2,
      seg = 2,
      caxislabels = c("", "50 %", "100 %"),
      axislabcol = "black",
      vlcex = 1.35,
      font.axis = 2
    )

    mtext(endpoint_title, side = 3, line = 1.6, cex = 1.9, font = 2)

  } else {
    par(mai = c(0.20, 0.20, 0.65, 0.20))

    radarchart(
      df_radar,
      axistype = 0,
      pcol = "darkred",
      pfcol = rgb(1, 0, 0, 0.30),
      plwd = 2,
      cglcol = "darkgrey",
      cglty = 3,
      cglwd = 1.0,
      seg = 2,
      vlabels = rep("", ncol(df_radar)),
      vlcex = 0.01,
      caxislabels = rep("", 3),
      axislabcol = rgb(0, 0, 0, 0)
    )

    mtext(endpoint_title, side = 3, line = 0.25, cex = 1.05, font = 2)
  }
}

counter <- 1

for (endpoint in endpoints_plot) {

  file_name <- paste0("fig4a_", counter, "_new.svg")

  if (counter == 1) {
    svg(file_name, width = 9.5, height = 9.5, bg = "transparent")
    draw_radar(endpoint, show_labels = TRUE)
  } else {
    svg(file_name, width = 8.5, height = 8.5, bg = "transparent")
    draw_radar(endpoint, show_labels = FALSE)
  }

  dev.off()

  uplcmd <- paste0("dx upload ", file_name, " --path UKBRISK/Plots/", file_name)
  system(uplcmd)

  counter <- counter + 1
}

## 4.3 Stacked Barplot

In [ ]:
library(RColorBrewer)

In [ ]:
palette <- brewer.pal(n = 7, name = "OrRd")


ggplot(summary_df2, aes(x = endpoint, y = percentage, fill = ts_cat)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = palette, guide = guide_legend(nrow = 1)) +  
  labs(title = "Stacked Barplot for 'ts_cat' Category Across Endpoints", 
       y = "Percentage Contribution") +   
  theme_minimal() +
  theme(
    panel.grid = element_blank(),               
    axis.line = element_line(size = 1),          
    axis.text.x = element_text(angle = 45, hjust = 1, size = 14),  
    axis.title.x = element_blank(),           
    axis.title.y = element_text(size = 16),      
    legend.position = "bottom",                 
    legend.title = element_blank(),             
    legend.text = element_text(size = 12),     
    legend.box = "horizontal",                   
    plot.margin = margin(20, 10, 10, 50),        
    plot.title = element_text(size = 20, face = "bold")  
  )


In [ ]:
ggsave(
  filename = "fig4b.svg",   
  width = 10, height = 8,   
  dpi = 1000,                
  bg = "transparent"        
)

system("dx upload fig4b.svg --path UKBRISK/Plots/fig4b.svg")


# 5. NHC: Predictive capacities, cost/gain, risk group reassignment

## 5.1 Discriminative Performance

### 5.1.1 DL bootstrapped files

In [ ]:
system("dx download 'UKBRISK/PlotData/absC_benchmarking_bootstrapped_nhc.tsv'")
plot_data_abs_nhc <- read.delim("absC_benchmarking_bootstrapped_nhc.tsv", sep = "\t")

In [ ]:
system("dx download 'UKBRISK/PlotData/deltaC_benchmarking_bootstrapped_nhc.tsv'")
plot_data_delta_nhc <- read.delim("deltaC_benchmarking_bootstrapped_nhc.tsv", sep = "\t")
plot_data_delta_nhc <- plot_data_delta_nhc %>%
  mutate(across(where(is.numeric), ~ . * -1))

### 5.1.2 Data Prep

In [ ]:
plot_data_abs_nhc <- plot_data_abs_nhc %>% filter(Model %in% c("nhc", "nhc_pmh_ts", "initial_pmh_ts_secondary_nhc_pmh_ts_5pct", "initial_pmh_ts_secondary_nhc_prs_metabolomics_pmh_ts_5pct"))
plot_data_abs_nhc$Model <- recode(plot_data_abs_nhc$Model, 
                          "nhc" = "1-Step NHS Health Check", 
                          "nhc_pmh_ts" = "1-Step NHS Health Check + PMH/Questionnaires", 
                          "initial_pmh_ts_secondary_nhc_pmh_ts_5pct" = "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk)", 
                          "initial_pmh_ts_secondary_nhc_prs_metabolomics_pmh_ts_5pct" = "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk, Omics for Attendees)")

In [ ]:
plot_data_delta_nhc <- plot_data_delta_nhc %>% filter(Model2 %in% c("nhc", "nhc_pmh_ts", "initial_pmh_ts_secondary_nhc_pmh_ts_5pct", "initial_pmh_ts_secondary_nhc_prs_metabolomics_pmh_ts_5pct"))
plot_data_delta_nhc$Model2 <- recode(plot_data_delta_nhc$Model2, 
                          "nhc" = "1-Step NHS Health Check", 
                          "nhc_pmh_ts" = "1-Step NHS Health Check + PMH/Questionnaires", 
                          "initial_pmh_ts_secondary_nhc_pmh_ts_5pct" = "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk)", 
                          "initial_pmh_ts_secondary_nhc_prs_metabolomics_pmh_ts_5pct" = "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk, Omics for Attendees)")

In [ ]:
plot_data_abs_nhc$Model <- factor(plot_data_abs_nhc$Model, levels = c(
    "1-Step NHS Health Check",
    "1-Step NHS Health Check + PMH/Questionnaires",
    "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk)",
    "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk, Omics for Attendees)"  
))

plot_data_delta_nhc$Model2 <- factor(plot_data_delta_nhc$Model2, levels = c(
    "1-Step NHS Health Check",
    "1-Step NHS Health Check + PMH/Questionnaires",
    "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk)",
    "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk, Omics for Attendees)" 
))

### 5.1.3 C Plots

In [ ]:
background_data <- data.frame(
  ymin = seq(0.5, length(levels(as.factor(plot_data_abs_nhc$Endpoint))) - 0.5, 2),
  ymax = seq(1.5, length(levels(as.factor(plot_data_abs_nhc$Endpoint))) + 0.5, 2),
  xmin = -Inf,
  xmax = Inf
)

In [ ]:
background_data <- data.frame(
  ymin = seq(0.5, length(unique(plot_data_abs_nhc$Endpoint)) - 0.5, 2),
  ymax = seq(1.5, length(unique(plot_data_abs_nhc$Endpoint)) + 0.5, 2),
  xmin = -Inf,
  xmax = Inf
)

plot_data_abs_nhc$Endpoint <- recode(plot_data_abs_nhc$Endpoint,
                                     "RD" = "Kidney disease",
                                     "CVD" = "Cardiovascular disease",
                                     "DM" = "Diabetes mellitus")

options(repr.plot.width = 20, repr.plot.height = 5)  


abs_nhc_plot1 <- ggplot(plot_data_abs_nhc, aes(x = C_Index, y = Endpoint, fill = Model, color = Model)) +
    geom_rect(data = background_data, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
            fill = "lightgray", alpha = 0.5, inherit.aes = FALSE) +
    geom_point(size = 5, shape = 21, stroke = 0.6, position = position_dodge(width = -0.5)) +
    geom_errorbarh(aes(xmin = Bottom95, xmax = Top95), height = 0.2, position = position_dodge(width = -0.5)) +
    scale_fill_manual(values = c("1-Step NHS Health Check" = "azure2", 
                               "1-Step NHS Health Check + PMH/Questionnaires" = "azure4", 
                               "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk)" = "#ACD8A7", 
                               "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk, Omics for Attendees)" = "#52A447")) +
    scale_color_manual(values = c("1-Step NHS Health Check" = "black", 
                                "1-Step NHS Health Check + PMH/Questionnaires" = "black", 
                                "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk)" = "black", 
                                "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk, Omics for Attendees)" = "black")) +
    theme_minimal() +
    labs(x = "Harrel's C", y = "") +
    theme(
    legend.position = "right",
    legend.title = element_text(size = 12),
    legend.text = element_text(size = 12),
    axis.line = element_line(size = 1),  
    axis.ticks = element_line(size = 1),  
    axis.text = element_text(size = 14),  
    axis.title.x = element_text(size = 14),  
    axis.title.y = element_text(size = 14)  
  )

abs_nhc_plot1

In [ ]:
plot_data_delta_nhc$Endpoint <- recode(plot_data_delta_nhc$Endpoint,
                                     "RD" = "Kidney disease",
                                     "CVD" = "Cardiovascular disease",
                                     "DM" = "Diabetes mellitus")

delta_nhc_plot_1 <- ggplot(plot_data_delta_nhc, aes(x = MeanDeltaC, y = Endpoint, fill = Model2)) +
    geom_rect(data = background_data, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
            fill = "lightgray", alpha = 0.5, inherit.aes = FALSE) +
    geom_errorbarh(aes(xmin = Bottom95, xmax = Top95), height = 0.2, position = position_dodge(width = -0.5)) +
    geom_vline(xintercept = 0, linetype = "dashed", color = "#FF9966", size = 1.2) +  
    geom_point(aes(fill = Model2), color = "black", size = 5, shape = 21, stroke = 0.6, position = position_dodge(width = -0.5)) +
    scale_fill_manual(values = c("1-Step NHS Health Check" = "azure2", 
                               "1-Step NHS Health Check + PMH/Questionnaires" = "azure4", 
                               "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk)" = "#ACD8A7", 
                               "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk, Omics for Attendees)" = "#52A447")) +
    scale_color_manual(values = c("1-Step NHS Health Check" = "black", 
                                "1-Step NHS Health Check + PMH/Questionnaires" = "black", 
                                "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk)" = "black", 
                                "2-Step NHS Health Check (No Needle Model-driven Invitation of > 5% Risk, Omics for Attendees)" = "black")) +
    theme_minimal() +
    labs(x = "ΔHarrell's C (Baseline: NHS Health Check)", y = "") +
    theme(
    legend.position = "none",
    legend.title = element_text(size = 12),
    legend.text = element_text(size = 12),
    axis.line = element_line(size = 1),  
    axis.ticks = element_line(size = 1),  
    axis.text = element_text(size = 14),  
    axis.title.x = element_text(size = 14),  
    axis.title.y = element_text(size = 14)  
  )

delta_nhc_plot_1


## 5.2 Density Plots


In [ ]:
install.packages("ggbreak")

In [ ]:
# Define a function to convert survival probabilities to risk probabilities
calculate_risk <- function(probs) {
    as.numeric(1 - probs)
}

# Initialize an empty data frame for combining
combined_df <- data.frame()

# Loop over each endpoint to create and save the density plot for the pmh_ts combination
for (endpoint in names(test_dataframes_absrisk_nhc_10y)) {
  # Get the train data for the endpoint
  df_test <- test_dataframes_absrisk_nhc_10y[[endpoint]]
  
  # Calculate the risk probabilities
  df_test$risk <- calculate_risk(df_test$pmh_ts)
  
  # Add the endpoint as a new column
  df_test$endpoint <- endpoint
  
  # Combine this endpoint's data into the combined data frame using bind_rows
  combined_df <- bind_rows(combined_df, df_test)
}

In [ ]:
endpoint_colors <- c(
  DM  = "#2C7BB6",  # deep blue
  RD  = "#4DAF4A",  # muted green
  CVD = "#F4A3A3"   # warm red
)

endpoint_labels <- c(
  DM  = "Diabetes",
  RD  = "Renal disease",
  CVD = "Cardiovascular disease"
)

In [ ]:
library(dplyr)
library(survival)
library(ggplot2)
library(ggbreak)

# ---- Base density plot (rebuild cleanly) ----
combined_df$endpoint <- factor(combined_df$endpoint, levels = c("DM", "RD", "CVD"))

plot <- ggplot(combined_df, aes(x = risk, fill = endpoint)) +
  geom_density(alpha = 0.5) +
  ggtitle("Density of 10-year Risk for NHS Health Check Endpoints (No Needle Model)") +
  xlab("Predicted 10-Year Risk (No Needle Model, %)") +
  ylab(NULL) +
  theme_minimal() +
  scale_x_continuous(
    limits = c(0, 1.0),
    breaks = c(seq(0, 0.20, by = 0.05), 1.0),
    labels = c(
      scales::percent(seq(0, 0.20, by = 0.05), accuracy = 1),
      "100%"
    )
  ) +
  scale_x_break(c(0.20, 0.95)) +
  scale_y_continuous(limits = c(0, 70)) +
  scale_fill_manual(values = endpoint_colors, labels = endpoint_labels) +
  geom_vline(xintercept = 0.05, color = "orange", linetype = "dashed", size = 1) +
  geom_vline(xintercept = 0.10, color = "red",    linetype = "dashed", size = 1) +
  theme(
    legend.position = "right",
    legend.title = element_text(size = 12),
    legend.text = element_text(size = 12),
    axis.line = element_line(size = 1),
    axis.ticks = element_line(size = 1),
    axis.text = element_text(size = 14),
    axis.title.x = element_text(size = 14),
    axis.title.y = element_text(size = 14),
    plot.title = element_text(size = 14),
    plot.margin = margin(t = 5, r = 50, b = 5, l = 5)
  ) +
  coord_cartesian(clip = "off")

# ---- Build unified follow-up + status columns ----
obs_df <- combined_df %>%
  mutate(
    fu = case_when(
      endpoint == "CVD" ~ as.numeric(CVD_followup),
      endpoint == "DM"  ~ as.numeric(DM_followup),
      endpoint == "RD"  ~ as.numeric(RD_followup),
      TRUE ~ NA_real_
    ),
    event = case_when(
      endpoint == "CVD" ~ as.integer(CVD_status),
      endpoint == "DM"  ~ as.integer(DM_status),
      endpoint == "RD"  ~ as.integer(RD_status),
      TRUE ~ NA_integer_
    )
  ) %>%
  filter(is.finite(risk), risk >= 0, risk <= 0.20, !is.na(fu), !is.na(event)) %>%
  group_by(endpoint) %>%
  mutate(decile = ntile(risk, 10)) %>%
  ungroup()

# ---- 10-year point (days vs years safe) ----
t0 <- if (max(obs_df$fu, na.rm = TRUE) > 100) 365.25 * 10 else 10

# ---- KM 10-year incidence by endpoint x decile (95% CI) ----
inc_dec <- obs_df %>%
  group_by(endpoint, decile) %>%
  group_modify(~{
    sf <- survfit(Surv(fu, event) ~ 1, data = .x, conf.int = 0.95)
    s  <- summary(sf, times = t0)
    tibble(
      x     = median(.x$risk, na.rm = TRUE),
      inc10 = 1 - s$surv[1],
      lo    = 1 - s$upper[1],
      hi    = 1 - s$lower[1]
    )
  }) %>%
  ungroup() %>%
  filter(is.finite(x), is.finite(inc10), is.finite(lo), is.finite(hi))

# ---- R² per endpoint ----
r2_df <- inc_dec %>%
  group_by(endpoint) %>%
  summarise(r2 = summary(lm(inc10 ~ x))$r.squared, .groups = "drop") %>%
  mutate(
    label = paste0(endpoint_labels[as.character(endpoint)],
                   ": R\u00B2 = ", format(round(r2, 3), nsmall = 3))
  ) %>%
  arrange(endpoint)

r2_df$y_pos <- seq(60, 45, length.out = nrow(r2_df))
r2_x <- 0.22  # put text into the break-gap (between 20% and 100%)

# ---- Secondary axis mapping (0–20% observed incidence) ----
y_top <- 70
inc_max <- 0.20
k <- y_top / inc_max

inc_dec <- inc_dec %>%
  mutate(y = inc10 * k, y_lo = lo * k, y_hi = hi * k)

# ---- Perfect calibration line ----
cal_line <- data.frame(x = seq(0, 0.20, length.out = 200)) %>%
  mutate(y = x * k)

# ---- Overlay: perfect calibration + 95% CI + points + R² outside ----
plot <- plot +
  geom_line(
    data = cal_line,
    aes(x = x, y = y),
    inherit.aes = FALSE,
    colour = "black",
    linetype = "dashed",
    linewidth = 0.7
  ) +
  geom_errorbar(
    data = inc_dec,
    aes(x = x, ymin = y_lo, ymax = y_hi, colour = endpoint),
    inherit.aes = FALSE,
    width = 0,
    linewidth = 0.6,
    alpha = 0.7
  ) +
  geom_line(
    data = inc_dec,
    aes(x = x, y = y, colour = endpoint, group = endpoint),
    inherit.aes = FALSE,
    linewidth = 1
  ) +
  geom_point(
    data = inc_dec,
    aes(x = x, y = y, fill = endpoint),
    inherit.aes = FALSE,
    shape = 21,
    colour = "black",
    stroke = 0.6,
    size = 2.6,
    alpha = 0.7
  ) +
  geom_text(
    data = r2_df,
    aes(x = r2_x, y = y_pos, label = label),
    inherit.aes = FALSE,
    hjust = 0,
    size = 3.6,
    colour = "black"
  ) +
  scale_colour_manual(values = endpoint_colors, labels = endpoint_labels) +
  scale_y_continuous(
    limits = c(0, y_top),
    name = "Density",
    sec.axis = sec_axis(
      ~ . / k,
      name = "Observed 10-year incidence (95% CI)",
      breaks = seq(0, 0.20, by = 0.05),
      labels = scales::percent_format(accuracy = 1)
    )
  )

plot

In [ ]:
library(dplyr)
library(ggplot2)
library(ggbreak)
library(scales)

to_risk <- function(s) 1 - as.numeric(s)

df_cvd <- final_abs_nhc_10y_test[["CVD"]]
df_rd  <- final_abs_nhc_10y_test[["RD"]]
df_dm  <- final_abs_nhc_10y_test[["DM"]]

baseline_col <- "nhc"
remote_col   <- "pmh_ts"
col_5  <- "initial_pmh_ts_secondary_nhc_pmh_ts_5pct"
col_10 <- "initial_pmh_ts_secondary_nhc_pmh_ts_10pct"

# ---- Join (same individuals) ----
cvd_keep <- df_cvd %>%
  select(eid, CVD_followup, CVD_status, pmh_ts, nhc, all_of(col_5), all_of(col_10)) %>%
  rename_with(~ paste0(.x, "__CVD"), -eid)

rd_keep <- df_rd %>%
  select(eid, RD_followup, RD_status, pmh_ts, nhc, all_of(col_5), all_of(col_10)) %>%
  rename_with(~ paste0(.x, "__RD"), -eid)

dm_keep <- df_dm %>%
  select(eid, DM_followup, DM_status, pmh_ts, nhc, all_of(col_5), all_of(col_10)) %>%
  rename_with(~ paste0(.x, "__DM"), -eid)

dat <- cvd_keep %>%
  inner_join(rd_keep, by = "eid") %>%
  inner_join(dm_keep, by = "eid")

# ---- 10-year events ----
max_fu <- max(dat$CVD_followup__CVD, dat$RD_followup__RD, dat$DM_followup__DM, na.rm = TRUE)
t0 <- if (max_fu > 100) 365.25 * 10 else 10

ev10 <- list(
  CVD = dat$CVD_status__CVD & dat$CVD_followup__CVD <= t0,
  RD  = dat$RD_status__RD  & dat$RD_followup__RD  <= t0,
  DM  = dat$DM_status__DM  & dat$DM_followup__DM  <= t0
)

# total events within 10y (denominator for bars)
total_events <- c(
  CVD = sum(ev10$CVD, na.rm = TRUE),
  RD  = sum(ev10$RD,  na.rm = TRUE),
  DM  = sum(ev10$DM,  na.rm = TRUE)
)

missed_ep <- function(ep, col) {
  pred <- dat[[paste0(col, "__", ep)]]
  sum(ev10[[ep]] & (to_risk(pred) < 0.10), na.rm = TRUE)
}

miss_base   <- c(CVD = missed_ep("CVD", baseline_col),
                 RD  = missed_ep("RD",  baseline_col),
                 DM  = missed_ep("DM",  baseline_col))

miss_5      <- c(CVD = missed_ep("CVD", col_5),
                 RD  = missed_ep("RD",  col_5),
                 DM  = missed_ep("DM",  col_5))

miss_10     <- c(CVD = missed_ep("CVD", col_10),
                 RD  = missed_ep("RD",  col_10),
                 DM  = missed_ep("DM",  col_10))

miss_remote <- c(CVD = missed_ep("CVD", remote_col),
                 RD  = missed_ep("RD",  remote_col),
                 DM  = missed_ep("DM",  remote_col))

# ---- Bars: Δ missed events as % of all events within 10y ----
bars <- bind_rows(
  data.frame(cutoff = 0.05, endpoint = names(miss_base),
             delta = (miss_5      - miss_base) / total_events[names(miss_base)]),
  data.frame(cutoff = 0.10, endpoint = names(miss_base),
             delta = (miss_10     - miss_base) / total_events[names(miss_base)]),
  data.frame(cutoff = 1.00, endpoint = names(miss_base),
             delta = (miss_remote - miss_base) / total_events[names(miss_base)])
) %>%
  mutate(endpoint = factor(endpoint, levels = c("DM","RD","CVD")))

# ---- Burden curve: % not requiring in-person ----
r_remote_max <- pmax(
  to_risk(dat[[paste0(remote_col, "__CVD")]]),
  to_risk(dat[[paste0(remote_col, "__RD")]]),
  to_risk(dat[[paste0(remote_col, "__DM")]]),
  na.rm = TRUE
)

cuts <- seq(0, 0.20, by = 0.001)
burden <- data.frame(
  cutoff = cuts,
  pct_not_inperson = vapply(cuts, function(c) mean(r_remote_max < c, na.rm = TRUE), numeric(1))
)
burden <- bind_rows(burden, data.frame(cutoff = 1.00, pct_not_inperson = 1))

# draw curve downward from 0
burden$y_curve <- -burden$pct_not_inperson

# ---- Plot ----
p_5D <- ggplot() +
  geom_area(
    data = burden,
    aes(x = cutoff, y = y_curve),
    fill = "grey80",
    alpha = 0.4
  ) +
  geom_line(
    data = burden,
    aes(x = cutoff, y = y_curve),
    colour = "black",
    linewidth = 0.6
  ) +
  geom_hline(yintercept = 0, linewidth = 0.9) +
  geom_col(
    data = bars,
    aes(x = cutoff, y = delta, fill = endpoint),
    position = position_dodge(width = 0.02),
    width = 0.015,
    colour = "black",
    linewidth = 0.6
  ) +
  scale_fill_manual(values = endpoint_colors, labels = endpoint_labels) +
  scale_x_continuous(
    breaks = c(0.05, 0.10, 1.00),
    labels = c("5%", "10%", "100% remote")
  ) +
  scale_y_continuous(
    limits = c(-1, 1),
    name = "Missed individuals (vs. the NHS Health Check)",
    labels = percent_format(accuracy = 1),
    sec.axis = sec_axis(
      ~ -.,
      name = "Individuals not requiring in-person assessment (cumulative)",
      labels = percent_format(accuracy = 1)
    )
  ) +
  scale_x_break(c(0.20, 0.95)) +
  labs(x = "Pre-screening cut-off (predicted 10-year risk)") +
  theme_minimal(base_size = 13) +
  theme(
    panel.grid.minor = element_blank(),
    axis.line.x = element_blank(),   # remove x-axis line
    axis.ticks.x = element_line(colour = "black", linewidth = 1),  # keep ticks
    axis.line.y = element_line(colour = "black", linewidth = 1),
    axis.ticks.y = element_line(colour = "black", linewidth = 1),
    legend.title = element_blank()
  )

p_5D

In [ ]:
library(ggplot2)
library(scales)
library(ggbreak)
library(ggh4x)

# ---- endpoint colors (lines/points/bars) ----
endpoint_colors <- c(
  DM  = "#2C7BB6",
  RD  = "#4DAF4A",
  CVD = "#E15759"
)

# ---- less saturated fills (top densities) ----
endpoint_fill_colors <- c(
  DM  = "#9EC5DE",
  RD  = "#B9E3B8",
  CVD = "#F3B4B4"
)

endpoint_labels <- c(
  DM  = "Diabetes mellitus",
  RD  = "Renal disease",
  CVD = "Cardiovascular disease"
)

# ---- x-break ----
x_break_gap <- c(0.20, 0.95)

# ---- force facet order (so per-facet y scales map correctly) ----
panel_top <- "A:  Predicted & Observed Risk"
panel_bot <- "B:  Screening Burden & Missed/Detected Events"
panel_levels <- c(panel_top, panel_bot)

dens_df   <- transform(combined_df, panel = factor(panel_top, levels = panel_levels))
cal_df    <- transform(inc_dec,     panel = factor(panel_top, levels = panel_levels))
line_df   <- transform(cal_line,    panel = factor(panel_top, levels = panel_levels))
burden_df <- transform(burden,      panel = factor(panel_bot, levels = panel_levels))
bars_df   <- transform(bars,        panel = factor(panel_bot, levels = panel_levels))

p_all <- ggplot() +
  # ================= TOP =================
  geom_density(
    data = dens_df,
    aes(x = risk, fill = endpoint),
    alpha = 0.35
  ) +
  geom_line(
    data = line_df,
    aes(x = x, y = y),
    linetype = "dashed",
    colour = "black",
    linewidth = 0.7
  ) +
  geom_errorbar(
    data = cal_df,
    aes(x = x, ymin = y_lo, ymax = y_hi, colour = endpoint),
    width = 0.003,
    linewidth = 0.6
  ) +
  geom_line(
    data = cal_df,
    aes(x = x, y = y, colour = endpoint),
    linewidth = 0.9
  ) +
  geom_point(
    data = cal_df,
    aes(x = x, y = y, colour = endpoint),
    size = 2.2
  ) +

  # ================= BOTTOM =================
  geom_area(
    data = burden_df,
    aes(x = cutoff, y = y_curve),
    fill = "grey80",
    alpha = 0.35
  ) +
  geom_line(
    data = burden_df,
    aes(x = cutoff, y = y_curve),
    colour = "grey35",
    linewidth = 0.6
  ) +
  geom_hline(
    data = burden_df,
    aes(yintercept = 0),
    linewidth = 0.9
  ) +
  geom_col(
    data = bars_df,
    aes(x = cutoff, y = delta, fill = endpoint),
    position = position_dodge(width = 0.02),
    width = 0.015,
    colour = "black",
    linewidth = 0.6
  ) +

  # ================= shared cutoffs =================
  geom_vline(xintercept = 0.05, colour = "orange", linetype = "dashed", linewidth = 1) +
  geom_vline(xintercept = 0.10, colour = "red",    linetype = "dashed", linewidth = 1) +

  # palettes
  scale_fill_manual(values = endpoint_fill_colors, labels = endpoint_labels) +
  scale_colour_manual(values = endpoint_colors, labels = endpoint_labels) +

  # x + break (NOTE: extend to 110% so x=1.00 bars are fully visible)
  scale_x_continuous(
    limits = c(0, 1.03),
    breaks = c(0, 0.05, 0.10, 0.15, 0.20, 1.00),
    labels = c("0%", "5%", "10%", "15%", "20%", "100%")
  ) +
  scale_x_break(x_break_gap) +

  # facets
  facet_wrap(~panel, ncol = 1, scales = "free_y") +

  # per-facet y scales (dense minor breaks for faint horizontal lines)
  ggh4x::facetted_pos_scales(
    y = list(
      scale_y_continuous(
        limits = c(0, y_top),
        breaks = pretty_breaks(n = 5),
        minor_breaks = seq(0, y_top, length.out = 25),
        name = "Density",
        sec.axis = sec_axis(~ . / k, name = NULL)
      ),
      scale_y_continuous(
        limits = c(-1, 0.5),
        breaks = pretty_breaks(n = 5),
        minor_breaks = seq(-1, 0.5, by = 0.05),
        name = "Missed individuals (vs. the NHS Health Check)",
        labels = percent_format(accuracy = 1),
        sec.axis = sec_axis(~ -., name = NULL)
      )
    )
  ) +

  labs(x = "Predicted 10-year risk (No Needle model)", y = NULL) +
  coord_cartesian(clip = "off") +
  theme_minimal(base_size = 13) +
  theme(
    # faint, dense horizontal lines
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    panel.grid.major.y = element_line(colour = "grey90", linewidth = 0.3),
    panel.grid.minor.y = element_line(colour = "grey95", linewidth = 0.2),

    # axis lines: left, right, bottom (NO top)
    axis.line = element_blank(),
    axis.line.y.left   = element_line(colour = "black", linewidth = 1),
    axis.line.y.right  = element_line(colour = "black", linewidth = 1),
    axis.line.x.bottom = element_line(colour = "black", linewidth = 1),
    axis.line.x.top    = element_blank(),

    # remove any top axis ticks/labels
    axis.text.x.top  = element_blank(),
    axis.ticks.x.top = element_blank(),

    # keep bottom ticks
    axis.ticks.x = element_line(colour = "black", linewidth = 1),
    axis.ticks.y = element_line(colour = "black", linewidth = 1),

    # remove built-in right labels/ticks (they show up near the break)
    axis.text.y.right  = element_blank(),
    axis.ticks.y.right = element_blank(),
    axis.title.y.right = element_blank(),

    legend.title = element_blank(),
    legend.position = "right",
    strip.text = element_text(face = "bold"),

    # extra room for manual right-axis labels
    plot.margin = margin(6, 70, 6, 8)
  )

p_all

In [ ]:
ggsave(
  filename = "fig5b_new.svg",
    plot = p_all, # Name of the file
  width = 20, height = 11,
    #dpi = 1000,
  bg = "transparent"        # Set background to transparent
)

system("dx upload fig5b_new.svg --path UKBRISK/Plots/fig5b_new.svg")

## 5.3 finalize fig 5 

In [ ]:
abs_nhc_plot1_no_legend <- abs_nhc_plot1 + theme(legend.position = "none")
combined_plots <- plot_grid(abs_nhc_plot1_no_legend, delta_nhc_plot_1, nrow = 1)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5)
print(combined_plots)

In [ ]:
ggsave(
  filename = "fig5b.svg",
    plot = combined_plots, # Name of the file
  width = 20, height = 5,
    #dpi = 1000,
  bg = "transparent"        # Set background to transparent
)

system("dx upload fig5b.svg --path UKBRISK/Plots/fig5b.svg")